In [1]:
from util import *

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# this block of codes comes from Nilavra Bhattacharya
# http://nbviewer.jupyter.org/urls/ivc.ischool.utexas.edu/VizWiz/extn/tutorial_skill_type.ipynb

df_train = pd.read_csv('../vizwiz_skill_typ_train.csv', skipinitialspace = True, engine='python')
df_val = pd.read_csv('../vizwiz_skill_typ_val.csv', skipinitialspace = True, engine='python')

In [4]:
# check class distributions
target_classes = ['TXT', 'OBJ', 'COL', 'CNT', 'OTH']
class_dist = []
for skill in target_classes:
    percent = len(df_train.loc[df_train[skill] == 1])/len(df_train)
    temp = {'skill': skill, '% of true': percent}
    class_dist.append(temp)
class_dist = pd.DataFrame(class_dist)
class_dist

,% of true,skill
0,0.561847,TXT
1,0.984652,OBJ
2,0.379564,COL
3,0.036513,CNT
4,0.001542,OTH


In [5]:
# correlations: OBJ is correlated with COL
df_train[['TXT', 'OBJ', 'COL', 'CNT', 'OTH']].corr()

,TXT,OBJ,COL,CNT,OTH
TXT,1.000000,0.031074,-0.479960,0.094344,-0.019298
OBJ,0.031074,1.000000,-0.035096,0.006068,0.004906
COL,-0.479960,-0.035096,1.000000,-0.106065,-0.016013
CNT,0.094344,0.006068,-0.106065,1.000000,-0.007650
OTH,-0.019298,0.004906,-0.016013,-0.007650,1.000000


In [ ]:
visual_features_train = aggregate_visual_features(df_train, 
                                                  cv_key=)

Downloading: "https://download.pytorch.org/models/vgg13-c768596a.pth" to /home/edithzeng/.torch/models/vgg13-c768596a.pth
100%|██████████| 532194478/532194478 [00:09<00:00, 54115929.19it/s]


In [ ]:
visual_features_train.to_csv('visual_features_train.csv')

In [5]:
print(visual_features_train.shape, df_train.shape)

(14269, 10) (14269, 19)
